## 3. Recommendation System

In [1]:
import pandas as pd
import numpy as np
import pickle

- 협업 필터링

In [2]:
import pickle 
with open('data/distances.pickle', 'rb') as f: 
    distances = pickle.load(f)

with open('data/indices.pickle', 'rb') as f: 
    indices = pickle.load(f)

with open('data/movie_pivot.pickle', 'rb') as f: 
    movie_pivot = pickle.load(f)

cf_df = pd.read_csv('data/cf_df.csv')

distances.shape, indices.shape, movie_pivot.shape, cf_df.shape

((1350, 11), (1350, 11), (1350, 143458), (17337458, 5))

- 콘텐츠 기반 필터링

In [3]:
with open('data/cosine_similarity_matrix.pickle', 'rb') as f: 
    cosine_similarity_matrix = pickle.load(f)

cb_df = pd.read_csv('data/cb_df.csv')

cosine_similarity_matrix.shape, cb_df.shape

((8807, 8807), (8807, 13))

> Merge

In [4]:
cf_copy = cf_df.copy()
cb_copy = cb_df.copy()

In [5]:
cf_copy = cf_copy.drop_duplicates(['Name']).reset_index(drop=True)
cf_copy = cf_copy.drop(columns=cf_copy.columns.difference(['Name']))

In [6]:
cf_copy['cf'] = 1
cf_copy

,Name,cf
0,Character,1
1,What the #$*! Do We Know!?,1
2,Screamers,1
3,7 Seconds,1
4,Immortal Beloved,1
...,...,...
1337,Wonder Boys,1
1338,Ned Kelly,1
1339,Club Dread,1
1340,Ju-on: The Grudge,1


In [7]:
cb_copy = cb_copy.drop(columns=cb_copy.columns.difference(['title'])).reset_index(drop=True)
cb_copy.columns = ['Name']

In [8]:
cb_copy['cb'] = 1
cb_copy

,Name,cb
0,Dick Johnson Is Dead,1
1,Blood & Water,1
2,Ganglands,1
3,Jailbirds New Orleans,1
4,Kota Factory,1
...,...,...
8802,Zodiac,1
8803,Zombie Dumb,1
8804,Zombieland,1
8805,Zoom,1


In [9]:
movie_df = pd.merge(cf_copy, cb_copy, how='outer', left_on='Name', right_on='Name')
movie_df.shape

(10052, 3)

In [10]:
movie_df.head()

,Name,cf,cb
0,Character,1.0,NaN
1,What the #$*! Do We Know!?,1.0,NaN
2,Screamers,1.0,NaN
3,7 Seconds,1.0,NaN
4,Immortal Beloved,1.0,NaN


In [11]:
movie_df['cf'].unique(), movie_df['cb'].unique() 

(array([ 1., nan]), array([nan,  1.]))

In [12]:
movie_df['cf'] = movie_df['cf'].fillna(0)
movie_df['cf'].unique()

array([1., 0.])

In [13]:
movie_df['cb'] = movie_df['cb'].fillna(0)
movie_df['cb'].unique()

array([0., 1.])

In [14]:
movie_df[(movie_df['cf'] == 1) & (movie_df['cb'] == 1)]

,Name,cf,cb
7,Something's Gotta Give,1.0,1.0
19,Dragonheart,1.0,1.0
22,Congo,1.0,1.0
30,Scandal,1.0,1.0
37,The Game,1.0,1.0
...,...,...,...
1246,Mars Attacks!,1.0,1.0
1251,City of Ghosts,1.0,1.0
1255,The Mirror Has Two Faces,1.0,1.0
1258,Life,1.0,1.0


In [15]:
movie_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10052 entries, 0 to 10051
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Name    10052 non-null  object 
 1   cf      10052 non-null  float64
 2   cb      10052 non-null  float64
dtypes: float64(2), object(1)
memory usage: 314.1+ KB


In [16]:
movie_df['cf'] = movie_df['cf'].astype('int8')
movie_df['cb'] = movie_df['cb'].astype('int8')
movie_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10052 entries, 0 to 10051
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Name    10052 non-null  object
 1   cf      10052 non-null  int8  
 2   cb      10052 non-null  int8  
dtypes: int8(2), object(1)
memory usage: 176.7+ KB


In [17]:
movie_df

,Name,cf,cb
0,Character,1,0
1,What the #$*! Do We Know!?,1,0
2,Screamers,1,0
3,7 Seconds,1,0
4,Immortal Beloved,1,0
...,...,...,...
10047,Zodiac,0,1
10048,Zombie Dumb,0,1
10049,Zombieland,0,1
10050,Zoom,0,1


In [18]:
movie_df[(movie_df['cf'] == 1) & (movie_df['cb'] == 1)][50:60]

,Name,cf,cb
656,BASEketball,1,1
660,Lock,1,1
663,Bad Boys,1,1
680,Biggie & Tupac,1,1
717,Scream 3,1,1
721,Center Stage,1,1
727,Scream,1,1
740,Like Water for Chocolate,1,1
744,A Cinderella Story,1,1
754,My Girl 2,1,1


In [20]:
# movie_df.to_csv('data/movie_df.csv', index=False)

In [19]:
movie_df = pd.read_csv('data/movie_df.csv')
movie_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10052 entries, 0 to 10051
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Name    10052 non-null  object
 1   cf      10052 non-null  int64 
 2   cb      10052 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 235.7+ KB


- Develop

In [22]:
# cf: Pirates of the Caribbean: The Curse of the Black Pearl, Love Actually
# cb: About Time, The Conjuring
# cf, cb: Scream 3, A Cinderella Story 

> index 검색

In [20]:
from recommendation_system import cf_search_movie
from recommendation_system import cb_get_index

while True: 
    # input movie name
    movie_name = input('영화 이름을 입력하세요. 종료 하고 싶으면 "종료"를 입력하세요')

    if movie_name == '종료':
        print('종료합니다.')
        break
    else:
        # reset index
        movie_index = {} 

        # 영화가 있으면
        if len(movie_df[movie_df['Name'] == movie_name]) == 1:
            search_movie = movie_df.loc[movie_df['Name'] == movie_name]
            if ((search_movie['cf'] == 1) & (search_movie['cb'] == 0)).values[0]:
                movie_index['cf'] = (cf_search_movie(movie_name, cf_df, movie_pivot))
                break
            elif ((search_movie['cf'] == 0) & (search_movie['cb'] == 1)).values[0]:
                movie_index['cb'] = (cb_get_index(movie_name, cb_df))
                break
            else:
                movie_index['cf'] = (cf_search_movie(movie_name, cf_df, movie_pivot))
                movie_index['cb'] = (cb_get_index(movie_name, cb_df))
                break

        # 영화가 없으면        
        else:
            print('해당 영화가 없습니다. 다시 입력해주세요.')
            continue

movie_index


{'cb': 3908}

In [21]:
len(movie_index)

1

> 추천

In [22]:
def print_cf(cf):
    print(f'영화: {cf[0][0]}')
    print('-------------------------------------------')
    print('협업 필터링 추천 영화')
    for i in range(1, len(cf)):
        print(f'[{i}] 영화: {cf[i][0]} | distance: {cf[i][1]}')

In [23]:
def print_cb(cb):
    cb = cb.reset_index(drop=True)
    print(f'영화: {cb["title"][0]}')
    print('-------------------------------------------')
    print('콘텐츠 기반 추천 영화')
    for i in range(1, len(cb)):
        print(f'[{i}] 영화: {cb["title"][i]} | cosine_similarity: {cb["cosine_similarity"][i]}')

In [24]:
from recommendation_system import collaborative_filtering
from recommendation_system import content_based_filtering

# reset
cf = 0
cb = 0
# 둘 다 있으면
if len(movie_index) == 2:
    cf = collaborative_filtering(movie_index['cf'], 4, cf_df, movie_pivot, indices, distances)
    cb = content_based_filtering(cb_df, cosine_similarity_matrix, movie_index['cb'], top_n=4)
    print_cf(cf)
    print('          ')
    print_cb(cb)
    

elif 'cf' in movie_index:
    cf = collaborative_filtering(movie_index['cf'], 7, cf_df, movie_pivot, indices, distances)
    print_cf(cf)

elif 'cb' in movie_index:
    cb = content_based_filtering(cb_df, cosine_similarity_matrix, movie_index['cb'], top_n=7)
    print_cb(cb)

영화: About Time
-------------------------------------------
콘텐츠 기반 추천 영화
[1] 영화: The Forty-Year-Old Version | cosine_similarity: 0.6428321078077688
[2] 영화: Otherhood | cosine_similarity: 0.6419816125432746
[3] 영화: Cake | cosine_similarity: 0.6418694700537597
[4] 영화: Oasis: Supersonic | cosine_similarity: 0.6406167713554606
[5] 영화: Amy | cosine_similarity: 0.6404810026612613
[6] 영화: La ley de Herodes | cosine_similarity: 0.638939800384434
